In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename));

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Моя задача заключается в том, чтобы опеределеить взаимосвязь между частотой  взаимодействие с профилем питомца (Pawpularity) и фотографиями этого профиля.


Показатель Pawpularity рассчитывается на основе статистики просмотра страниц каждого профиля питомца на страницах списка с использованием алгоритма, который нормализует данные о трафике на разных страницах, платформах (веб- и мобильных) и различных показателях.
Повторяющиеся клики, доступы к поисковым роботам и спонсируемые профили исключаются из анализа.

Используемые данные

train/ - Папка, содержащая фотографии обучающего набора формы {id}.jpg, где {id} - уникальный идентификатор профиля питомца.

train.csv - Метаданные (описанные ниже) для каждой фотографии в обучающем наборе, а также для цели, оценки Pawpularity фотографии. В столбце Id указывается уникальный идентификатор профиля питомца фотографии, соответствующий имени файла фотографии.

Особенности¶

Focus - Питомец выделяется на незагроможденном фоне, не слишком близко / далеко.

Eyes - Оба глаза смотрят вперед или почти вперед, по крайней мере, 1 глаз / зрачок достаточно ясны..

Face - Прилично чистое лицо, обращенное вперед или почти спереди

Near - Один питомец занимает значительную часть фотографии (примерно более 50% ширины или высоты фотографии).

Action - Домашнее животное в середине действия (например, прыжки).

Accessory - Прилагаемый физический или цифровой аксессуар / реквизит (например, игрушка, цифровая наклейка), за исключением ошейника и поводка.

Group - На фото больше 1 питомца.

Collage - Фотография с цифровой ретушью (т.е. с цифровой фоторамкой, комбинация нескольких фотографий).

Human - Естьли чловек на фотографии.

Occlusion - Конкретные нежелательные объекты, блокирующие часть домашнего животного (например, человека, клетку или забор). Обратите внимание, что не все * блокирующие объекты считаются закрытыми.

Info - Пользовательский текст или метки (например, имя домашнего животного, описание).

Blur - Заметно не в фокусе или шумно, особенно для глаз и лица питомца. Для записей с размытым изображением столбец “Глаза” всегда имеет значение 0.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
from PIL import Image

In [ ]:
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

Посмотрим на обучающий датасет

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")

In [ ]:
train.head()

In [ ]:
train.describe()


Также полезно сначала просмотреть распределение целевой переменной, когда вы впервые начинаете анализировать набор данных. 
В нашем случае это показатель Pawpularity, который находится в диапазоне 1-100. Один из способов сделать это - с помощью простой гистограммы.


Посмотрим на распределение Pawpularity

In [ ]:
#fig, ax = plt.subplots(1,1, figsize=(10, 5))
#sns.histplot(x='Pawpularity', data=train, ax=ax)
#ax.set_title('Pawpularity Distribution', fontsize=20)
#ax.set_ylim(0, 700)
#plt.legend()
#plt.show()

In [ ]:
train[['Pawpularity']].describe()

In [ ]:
sns.set(rc={'figure.figsize':(10,5), "lines.linewidth": 2.5})
sns.distplot(train["Pawpularity"], label="Pawpularity")
plt.axvline(train['Pawpularity'].mean(), c='red', ls='-', lw=1, label='Mean Pawpularity')
plt.axvline(train['Pawpularity'].median(),c='blue',ls='-',lw=1, label='Median Pawpularity')
plt.legend()
plt.show()

In [ ]:
train['Pawpularity'].mean()

In [ ]:
train[['Pawpularity']].describe()

На границах гистограммы есть скачки. Предположу, что это выбросы, поэтому отброшу их

In [ ]:
train = train.loc[((train["Pawpularity"]>3) & (train["Pawpularity"]<100))]
sns.set(rc={'figure.figsize':(10,5), "lines.linewidth": 2.5})
sns.distplot(train["Pawpularity"], label="Pawpularity")

In [ ]:
train['Pawpularity'].mean()

In [ ]:
sns.set(rc={'figure.figsize':(2.5,2)})

sns.set_style("white")
feature_variables = train.columns.values.tolist()
for variable in feature_variables[1:-1]:
    fig, ax = plt.subplots(1)
    sns.boxplot(data=train, x=variable, y='Pawpularity')
    fig.show()
#plt.subplots_adjust(wspace = 0.3, hspace = 0.3)


Построив диаграмму "ящик с усами" для каждого признака, мы можем сделать вывод,что в каждом случае распределения животных с каким-либо специфическим признаком почти не оказывает никакого влияния на значение Pawpularity.

Посмотрим также распределение оценок pawpularity по каждой переменной(особенности) объекта.
Оценки Pawpularity построены по осям y и 0 и 1 для каждой переменной объекта по осям x для ящика с усами. Для гистограмм строены pawpularity по осям x и подсчитываем 0 и 1 для каждого показателя Pawpularity. Это может помочь определить влияют ли 0 или 1 в каждой переменной характеристики на показатели Pawpularity.

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
feature_variables = train.columns.values.tolist()

for variable in feature_variables[1:-1]:
    fig, ax = plt.subplots(1,2)
    sns.boxplot(data=train, x=variable, y='Pawpularity', ax=ax[0])
    sns.histplot(train, x="Pawpularity", hue=variable, kde=True, ax=ax[1])
    plt.suptitle(variable, fontsize=20)
    fig.show()

Мы уже выяснили,что ящики с усами почти идентичны. И любому алгоритму будет сложно использовать эти данные для прогнозирования Pawpularity с использованием этой функции. Что касается гистограммы, то обратим внимание, что для этой функции существует гораздо больше значений 1, чем 0, но опять же распределение очень похоже.

**Вывод:**
Вы не всегда можно сказать, просто взглянув на диаграмму, можно ли построить модель с высокой степенью прогнозирования, но интуиция подсказывает мне, что модели, основанные на этих диаграммах, не будут отличными. Причина в том, что распределение оценок pawpularity очень похоже для каждой переменной и класса. Похоже, что изменение функций не так сильно влияет на оценки pawpularity. Это может означать, что для прогнозирования потребуется использовать изображения, а не метаданные из файлов ".csv".